In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java (Windows)
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-11-29 05:00:15--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.4’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-11-29 05:00:16 (6.08 MB/s) - ‘postgresql-42.2.9.jar.4’ saved [914037/914037]



In [3]:
# Start Spark session 
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MusicalInstrument").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("header",True) \
                    .option("sep", "\t") \
                    .option("multiLine", True) \
                    .option("quote","\"") \
                    .option("escape","\"") \
                    .option("ignoreTrailingWhiteSpace", True) \
                    .csv(SparkFiles.get("amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"))

In [5]:
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

In [6]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
df.columns

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

In [8]:
# Count the number of records (rows) in the dataset.
print(f"Number of records in the dataset is {df.count()}.")

Number of records in the dataset is 904040.


In [9]:
#drop duplicates and incomplete rows
df = df.dropna()
df = df.dropDuplicates()
#show total counts of reviews after dropping duplicates and incomplete rows
print(df.count())

903919


In [10]:
# Transform the dataset to fit the tables in the schema file.
vine_reviews = df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_reviews.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R10054OAXKUYEW|          5|            0|          1|   N|
|R1029CP64GCQVU|          5|            0|          0|   N|
|R102H3GGAZCZCF|          5|            5|         10|   N|
| R102SE89P3L1Y|          5|            1|          1|   N|
|R1037LO2K5NA94|          4|            0|          0|   N|
|R103XZMKT84REI|          5|            0|          0|   N|
|R105WD1IDQJPBN|          5|            7|          7|   N|
|R106H5VHYJ4BR1|          4|            0|          0|   N|
|R1085A4HD07EXZ|          4|            4|          4|   N|
|R108BDPC3QH8S2|          5|            1|          2|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [11]:
#select vine reviews
from pyspark.sql.functions import col
vine_reviews_yes = vine_reviews.filter(col("vine")=="Y")
#show the total count of vine reviews
print(vine_reviews_yes.count())
vine_reviews_yes.show(10)

2286
+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R23BVI0C1F4AHH|          4|            0|          2|   Y|
|R313DYY30GSEZH|          5|            0|          1|   Y|
|R3EWWMI9D0MWCL|          5|            9|         10|   Y|
|R21JA87N0ELCXP|          5|            0|          0|   Y|
|R35KNKK5PDGGO5|          4|            5|          5|   Y|
|R24APU51KMB4NT|          4|            0|          0|   Y|
|R1VWOIXRD8K422|          4|            0|          0|   Y|
|R171NW7T8LARPT|          5|            6|          9|   Y|
|R292O20PS0KPNS|          5|            0|          0|   Y|
|R26SH1ROEYI1R1|          5|            0|          0|   Y|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [12]:
vine_reviews_yes.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)



In [14]:
# Load the DataFrames that correspond to tables into an RDS instance. 
#endpoint and setup a new database in postgres aws cloud database
db_url = "jdbc:postgresql://<AWS RDS Endpoint>/<database name>"
config = {
    "user":"<user name>",
    "password":"<password>",
    "driver":"org.postgresql.Driver"
}

In [15]:
vine_reviews_yes.write.jdbc(url=db_url,table="musical_instrument_review",mode="append",properties=config)